# Warning:

This service is largely unused, and will be rescoped in the future. As such, both the service and this test suite are somewhat outdated and brittle. These tests should be tidied up as changes occur to HGA to reflect the new scope and capabilities of the service. Initial changes will likely include the removal of browse image generation and netCDF4 handling.

# Harmony GDAL Adapter (HGA) regression tests

The scope of this Jupyter notebook is to run a suite of regression tests against sample collections for the Harmony GDAL Adapter (HGA). These sample collections are primarily from the Alaska Satellite Facility (ASF), including:

* [ALOS AVNIR OBS ORI](https://cmr.earthdata.nasa.gov/search/concepts/C1808440897-ASF.html)
* [UAVSAR POLSAR Pauli](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1244141264-EEDTEST.html)

**NOTE:**

Previously, this notebook also contained tests for the [Sentinel 1 Interferograms collection](https://cmr.earthdata.nasa.gov/search/concepts/C1595422627-ASF.html). These tests were removed as HGA alone no longer supports netCDF4 input files (or netCDF4 inputs contained in a zip file). If a future service chain is created with net2cog as a preceding step to HGA, then those tests could represent a good baseline for a regression test suite of that service chain. Those tests can be found in [this version of the notebook](https://github.com/nasa/harmony-regression-tests/blob/68fd8b684d22937933260825f1738aa2f9ed9600/test/hga/HGA_Regression.ipynb).

## Set the Harmony environment:

The cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

The default value is for the UAT environment. When using this notebook there are two ways to use the non-default environment:

* Run this notebook in a local Jupyter notebook server and change the value of `harmony_host_url` in the cell below to the value for the environment you require from the above list.
* Use the `run_notebooks.sh` script, which requires you to declare an environment variable `HARMONY_HOST_URL`. Set that environment variable to the value above that corresponds to the environment you want to test. That environment variable will take precedence over the default value in the cell below.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-hga`

A `.netrc` file must also be located in the `test` directory of this repository.

### Import required packages:

In [ ]:
from datetime import datetime

# TODO: Enable import when earthdata-hashdiff is included in environment.yml
# from earthdata_hashdiff import nc4_matches_reference_hash_file
from harmony import BBox, Client, Collection, Environment, Request

from test_utilities import (
    check_request_output,
    clean_test_artefacts,
    get_test_granule_id,
    make_request_and_download_result,
)

### Set up environment dependent variables:

This includes the Harmony `Client` object and `Collection` objects for each of the collections for which there are regression tests. The local, SIT and UAT Harmony instances all utilise resources from CMR UAT, meaning any non-production environment will use the same resources.

When adding a production entry to the dictionary below, the collection instances can be included directly in the production dictionary entry, as they do not need to be shared.

In [ ]:
non_production_collections = {
    'avnir_collection': Collection(id='C1244141281-EEDTEST'),
    'sentinel_collection': Collection(id='C1244141250-EEDTEST'),
    'uavsar_collection': Collection(id='C1244141264-EEDTEST'),
    'mur_collection': Collection(id='C1238621141-POCLOUD'),
}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        'env': Environment.UAT,
        **non_production_collections,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        'env': Environment.SIT,
        **non_production_collections,
    },
    'http://localhost:3000': {'env': Environment.LOCAL, **non_production_collections},
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])

## AVNIR

The tests below will use the ALOS AVNIR OBS ORI collection (for UAT this is mirrored in the EEDTEST CMR environment). There are no reference images provided for this collection, as they would each be large (tens of MB).

### AVNIR test case 1:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_directory_one = 'avnir_test_one'

    avnir_request_one = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
    )

    avnir_output_one = make_request_and_download_result(
        harmony_client, avnir_request_one, avnir_directory_one
    )

    avnir_variable_base_one = 'ALAV2A237403590-OORIRFU'

    avnir_variables_one = {
        f'IMG-01-{avnir_variable_base_one}_000_band_1',
        f'IMG-02-{avnir_variable_base_one}_000_band_1',
    }

    expected_avnir_results_one = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': 'WGS 84 / UTM zone 22N',
        'proj_epsg': '32622',
        'spatial_extent': [-0.05, 0.3, -50.99, -50.75],
        'variables': avnir_variables_one,
        'width': 3410,
        'height': 3238,
    }

    check_request_output(avnir_output_one[0], expected_avnir_results_one)
    clean_test_artefacts(avnir_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 2:

This test case combines a bounding box spatial subset across the Prime Meridian with a variable subset of bands 3 and 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1817336128-ASF', 'G1244144599-EEDTEST'
    )

    avnir_directory_two = 'avnir_test_two'

    avnir_request_two = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-0.15, 14.8, 0.15, 14.9),
        variables=['Band3', 'Band4'],
        format='image/tiff',
        granule_id=granule_id,
    )

    avnir_output_two = make_request_and_download_result(
        harmony_client, avnir_request_two, avnir_directory_two
    )

    avnir_variable_base_two = 'ALAV2A264953300-OORIRFU'

    avnir_variables_two = {
        f'IMG-03-{avnir_variable_base_two}_000_band_1',
        f'IMG-04-{avnir_variable_base_two}_000_band_1',
    }

    expected_avnir_results_two = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': 'WGS 84 / UTM zone 30N',
        'proj_epsg': '32630',
        'spatial_extent': [14.74, 14.96, -0.14, 0.14],
        'variables': avnir_variables_two,
        'width': 3392,
        'height': 1762,
    }

    check_request_output(avnir_output_two[0], expected_avnir_results_two)
    clean_test_artefacts(avnir_directory_two)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 3:

This test combines a bounding box spatial subset across the anti-meridian with a variable subset for bands 1 and 2.  

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1835935580-ASF', 'G1244144598-EEDTEST'
    )

    avnir_directory_three = 'avnir_test_three'

    avnir_request_three = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(179.75, 66.5, -179.85, 66.75),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
    )

    avnir_output_three = make_request_and_download_result(
        harmony_client, avnir_request_three, avnir_directory_three
    )

    avnir_variable_base_three = 'ALAV2A239652240-OORIRFU'

    avnir_variables_three = {
        f'IMG-01-{avnir_variable_base_three}_002_band_1',
        f'IMG-02-{avnir_variable_base_three}_002_band_1',
    }

    expected_avnir_results_three = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': 'WGS 84 / UTM zone 1N',
        'proj_epsg': '32601',
        'spatial_extent': [66.44, 66.81, 179.85, -179.95],
        'variables': avnir_variables_three,
        'width': 2694,
        'height': 3250,
    }

    check_request_output(avnir_output_three[0], expected_avnir_results_three)
    clean_test_artefacts(avnir_directory_three)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 4:

This test case combines a bounding box spatial subset in the Northern Hemisphere with a variable subset requesting only band 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 1 output band.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1849288272-ASF', 'G1244144595-EEDTEST'
    )

    avnir_directory_four = 'avnir_test_four'

    avnir_request_four = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-99.15, 36.9, -98.85, 37.1),
        variables=['Band4'],
        format='image/tiff',
        granule_id=granule_id,
    )

    avnir_output_four = make_request_and_download_result(
        harmony_client, avnir_request_four, avnir_directory_four
    )

    avnir_variable_base_four = 'ALAV2A278852850-OORIRFU'
    avnir_variables_four = {f'IMG-04-{avnir_variable_base_four}_000_band_1'}

    expected_avnir_results_four = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': 'WGS 84 / UTM zone 14N',
        'proj_epsg': '32614',
        'spatial_extent': [36.85, 37.15, -99.13, -98.87],
        'variables': avnir_variables_four,
        'width': 3108,
        'height': 2774,
    }

    check_request_output(avnir_output_four[0], expected_avnir_results_four)
    clean_test_artefacts(avnir_directory_four)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 5:

This test case requests a bounding box spatial subsest in the Northern Hemisphere. No variable subset is specified, so the request will retrieve all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all variables and expect 4 output variables.
This output is identical to the output if each band is requested e.g. adding `variables=['Band1','Band2', 'Band3', 'Band4']` to the Request.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1849288272-ASF', 'G1244144595-EEDTEST'
    )

    avnir_directory_five = 'avnir_test_five'

    avnir_request_five = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-99.2, 36.95, -98.9, 37.05),
        format='image/tiff',
        granule_id=granule_id,
    )

    avnir_output_five = make_request_and_download_result(
        harmony_client, avnir_request_five, avnir_directory_five
    )

    avnir_variable_base_five = 'ALAV2A278852850-OORIRFU'

    avnir_variables_five = {
        f'IMG-03-{avnir_variable_base_five}_000_band_1',
        f'IMG-02-{avnir_variable_base_five}_000_band_1',
        f'IMG-04-{avnir_variable_base_five}_000_band_1',
        f'IMG-01-{avnir_variable_base_five}_000_band_1',
    }

    expected_avnir_results_five = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 4,
        'proj_cs': 'WGS 84 / UTM zone 14N',
        'proj_epsg': '32614',
        'spatial_extent': [36.9, 37.1, -99.18, -98.92],
        'variables': avnir_variables_five,
        'width': 2853,
        'height': 1693,
    }

    check_request_output(avnir_output_five[0], expected_avnir_results_five)
    clean_test_artefacts(avnir_directory_five)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 6:

This test combines a bounding box spatial subset in the Southern Hemisphere with a variable subset requesting bands 1 and 3 and specifies the height and width of the output bands.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1835549401-ASF', 'G1244144601-EEDTEST'
    )

    avnir_directory_six = 'avnir_test_six'

    avnir_request_six = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(18.07, -26.1, 18.25, -25.8),
        variables=['Band1', 'Band3'],
        format='image/tiff',
        granule_id=granule_id,
        height=30,
        width=75,
    )

    avnir_output_six = make_request_and_download_result(
        harmony_client, avnir_request_six, avnir_directory_six
    )

    avnir_variable_base_six = 'ALAV2A151894120-OORIRFU'

    avnir_variables_six = {
        f'IMG-01-{avnir_variable_base_six}_000_band_1',
        f'IMG-03-{avnir_variable_base_six}_000_band_1',
    }

    expected_avnir_results_six = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': 'WGS 84 / UTM zone 34S',
        'proj_epsg': '32734',
        'spatial_extent': [-26.13, -25.77, 18.07, 18.25],
        'variables': avnir_variables_six,
        'width': 75,
        'height': 30,
    }

    check_request_output(avnir_output_six[0], expected_avnir_results_six)
    clean_test_artefacts(avnir_directory_six)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 7:

This test combines a bounding box spatial subset (with negative longitude values) with all bands being explicitly specified alongside the height and width of the output array.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1906378530-ASF', 'G1244144603-EEDTEST'
    )

    avnir_directory_seven = 'avnir_test_seven'

    avnir_request_seven = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-97.65, 49.65, -97.35, 49.95),
        variables=['Band1', 'Band2', 'Band3', 'Band4'],
        granule_id=granule_id,
        height=45,
        width=90,
        format='image/tiff',
    )

    avnir_output_seven = make_request_and_download_result(
        harmony_client, avnir_request_seven, avnir_directory_seven
    )

    avnir_variable_base_seven = 'ALAV2A277832590-OORIRFU'

    avnir_variables_seven = {
        f'IMG-01-{avnir_variable_base_seven}_000_band_1',
        f'IMG-02-{avnir_variable_base_seven}_000_band_1',
        f'IMG-04-{avnir_variable_base_seven}_000_band_1',
        f'IMG-03-{avnir_variable_base_seven}_000_band_1',
    }

    expected_avnir_results_seven = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 4,
        'proj_cs': 'WGS 84 / UTM zone 14N',
        'proj_epsg': '32614',
        'spatial_extent': [49.6, 49.99, -97.64, -97.36],
        'variables': avnir_variables_seven,
        'width': 90,
        'height': 45,
    }

    check_request_output(avnir_output_seven[0], expected_avnir_results_seven)
    clean_test_artefacts(avnir_directory_seven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 8:

This test requests a bounding box spatial subset, where the bounding box is larger than granule itself. No variables are specified, meaning the request should return all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813329318-ASF', 'G1244144596-EEDTEST'
    )

    avnir_directory_eight = 'avnir_test_eight'

    avnir_request_eight = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-79.9, 9.2, -78.9, 10.3),
        granule_id=granule_id,
        format='image/tiff',
    )

    avnir_output_eight = make_request_and_download_result(
        harmony_client, avnir_request_eight, avnir_directory_eight
    )

    avnir_variable_base_eight = 'ALAV2A150623400-OORIRFU'

    avnir_variables_eight = {
        f'IMG-01-{avnir_variable_base_eight}_000_band_1',
        f'IMG-02-{avnir_variable_base_eight}_000_band_1',
        f'IMG-04-{avnir_variable_base_eight}_000_band_1',
        f'IMG-03-{avnir_variable_base_eight}_000_band_1',
    }

    expected_avnir_results_eight = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 4,
        'proj_cs': 'WGS 84 / UTM zone 17N',
        'proj_epsg': '32617',
        'spatial_extent': [9.4, 10.24, -79.7, -79.2],
        'variables': avnir_variables_eight,
        'width': 7215,
        'height': 8000,
    }

    check_request_output(avnir_output_eight[0], expected_avnir_results_eight)
    clean_test_artefacts(avnir_directory_eight)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 9:

This test combines a bounding box spatial subset with a temporal range subset. All variables are explicitly specified. No granules are specified in this request, meaning the results will be determined by those granules that match the spatial and temporal criteria.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are requesting "all" variables and expecting to retrieve all 4 output bands.


In [ ]:
if environment_information is not None:
    avnir_directory_nine = 'avnir_test_nine'

    avnir_request_nine = Request(
        collection=environment_information['avnir_collection'],
        variables=['all'],
        spatial=BBox(18.4, -26.1, 18.6, -26.0),
        format='image/tiff',
    )

    avnir_output_nine = make_request_and_download_result(
        harmony_client, avnir_request_nine, avnir_directory_nine
    )

    avnir_variable_base_nine = 'ALAV2A151894120-OORIRFU'

    avnir_variables_nine = {
        f'IMG-01-{avnir_variable_base_nine}_000_band_1',
        f'IMG-02-{avnir_variable_base_nine}_000_band_1',
        f'IMG-03-{avnir_variable_base_nine}_000_band_1',
        f'IMG-04-{avnir_variable_base_nine}_000_band_1',
    }

    expected_avnir_results_nine = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 4,
        'proj_cs': 'WGS 84 / UTM zone 34S',
        'proj_epsg': '32734',
        'spatial_extent': [-26.13, -25.96, 18.41, 18.59],
        'variables': avnir_variables_nine,
        'width': 2195,
        'height': 1516,
    }

    check_request_output(avnir_output_nine[0], expected_avnir_results_nine)
    clean_test_artefacts(avnir_directory_nine)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 10:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. No format is specified, such that the default of a GeoTIFF output should be returned.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_directory_ten = 'avnir_test_ten'

    avnir_request_ten = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        variables=['Band1', 'Band2'],
        granule_id=granule_id,
    )

    avnir_output_ten = make_request_and_download_result(
        harmony_client,
        avnir_request_ten,
        avnir_directory_ten,
    )

    avnir_variable_base_ten = 'ALAV2A237403590-OORIRFU'

    avnir_variables_ten = {
        f'IMG-01-{avnir_variable_base_ten}_000_band_1',
        f'IMG-02-{avnir_variable_base_ten}_000_band_1',
    }

    expected_avnir_results_ten = {
        'authority': 'EPSG',
        'cs': 'Projected',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': 'WGS 84 / UTM zone 22N',
        'proj_epsg': '32622',
        'spatial_extent': [-0.05, 0.3, -50.99, -50.75],
        'variables': avnir_variables_ten,
        'width': 3410,
        'height': 3238,
    }

    check_request_output(avnir_output_ten[0], expected_avnir_results_ten)
    clean_test_artefacts(avnir_directory_ten)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 11:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. A netCDF4 output format is specified.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

**Note: earthdata-hashdiff comparison is disabled because it requires Python 3.11, and that is incompatible with the version of GDAL used for GeoTIFF comparisons. This test is retained to ensure the request runs without error.**

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1813212660-ASF', 'G1244144597-EEDTEST'
    )

    avnir_directory_eleven = 'avnir_test_eleven'

    avnir_request_eleven = Request(
        collection=environment_information['avnir_collection'],
        spatial=BBox(-51.0, -0.05, -50.75, 0.25),
        format='application/x-netcdf4',
        granule_id=granule_id,
    )

    avnir_output_eleven = make_request_and_download_result(
        harmony_client,
        avnir_request_eleven,
        avnir_directory_eleven,
    )

    # assert nc4_matches_reference_hash_file(
    #     avnir_output_eleven[0], 'reference_images/avnir_reference_eleven.nc4'
    # ), 'Test output does not match reference file, AVNIR netCDF4 output'
    clean_test_artefacts(avnir_directory_eleven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

## UAVSAR POLSAR Pauli

The tests below will use the UAVSAR POLSAR Pauli collection (for UAT this is mirrored in the EEDTEST CMR environment).

The UAVSAR POLSAR data consists of 3 variables of data where each band represents a science variable (e.g., "HH minus VV, Red Band", "HV,Green Band" and "HH plus VV, Blue Band").

### UAVSAR test case 1:

This test combines a bounding box spatial subset with a variable subset (`Band1`).



In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_one = 'uavsar_test_one'

    uavsar_request_one = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_one = make_request_and_download_result(
        harmony_client, uavsar_request_one, uavsar_directory_one
    )

    uavsar_variable_base_one = 'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band1_subsetted__Band1__subsetted_band'

    uavsar_variables_one = {f'{uavsar_variable_base_one}_1'}

    expected_uavsar_results_one = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_one.tiff',
        'spatial_extent': [64.0, 64.1, -145.75, -145.7],
        'variables': uavsar_variables_one,
        'width': 450,
        'height': 1800,
    }

    check_request_output(uavsar_output_one[0], expected_uavsar_results_one)
    clean_test_artefacts(uavsar_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 2:

This test combines a bounding box spatial subset with a variable subset (`Band2`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_two = 'uavsar_test_two'

    uavsar_request_two = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.9, 63.75, -145.8, 63.80),
        variables=['Band2'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_two = make_request_and_download_result(
        harmony_client, uavsar_request_two, uavsar_directory_two
    )

    uavsar_variables_two = {
        'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band2_subsetted__Band2__subsetted_band_1'
    }

    expected_uavsar_results_two = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_two.tiff',
        'spatial_extent': [63.75, 63.80, -145.9, -145.8],
        'variables': uavsar_variables_two,
        'width': 900,
        'height': 900,
    }

    check_request_output(uavsar_output_two[0], expected_uavsar_results_two)
    clean_test_artefacts(uavsar_directory_two)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 3:

This test combines a bounding box spatial subset with a variable subset (`Band3`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV").

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_three = 'uavsar_test_three'

    uavsar_request_three = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.85, 63.85, -145.80, 63.90),
        variables=['Band3'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_three = make_request_and_download_result(
        harmony_client, uavsar_request_three, uavsar_directory_three
    )

    uavsar_variables_three = {
        'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band3_subsetted__Band3__subsetted_band_1'
    }

    expected_uavsar_results_three = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_three.tiff',
        'spatial_extent': [63.85, 63.90, -145.85, -145.80],
        'variables': uavsar_variables_three,
        'width': 450,
        'height': 900,
    }

    check_request_output(uavsar_output_three[0], expected_uavsar_results_three)
    clean_test_artefacts(uavsar_directory_three)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 4:

This test combines a bounding box spatial subset with a variable subset for multiple variables (`Band1` and `Band2`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") this test requests 2 bands. 

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1366089385-ASF', 'G1244144737-EEDTEST'
    )

    uavsar_directory_four = 'uavsar_test_four'

    uavsar_request_four = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-123.39, 40.05, -123.34, 40.06),
        variables=['Band1', 'Band2'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_four = make_request_and_download_result(
        harmony_client, uavsar_request_four, uavsar_directory_four
    )

    uavsar_variables_four = {
        'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__Band1__subsetted_band_1',
        'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__Band2__subsetted_band_1',
    }

    expected_uavsar_results_four = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 2,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_four.tiff',
        'spatial_extent': [40.05, 40.06, -123.39, -123.34],
        'variables': uavsar_variables_four,
        'width': 900,
        'height': 180,
    }

    check_request_output(uavsar_output_four[0], expected_uavsar_results_four)
    clean_test_artefacts(uavsar_directory_four)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 5:

This test performs a bounding box spatial subset, while requesting all variables.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., \"HH - VV\"), so the output is expected to have a total of 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1366089385-ASF', 'G1244144737-EEDTEST'
    )

    uavsar_directory_five = 'uavsar_test_five'

    uavsar_request_five = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-123.69, 39.95, -123.59, 40.0),
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_five = make_request_and_download_result(
        harmony_client, uavsar_request_five, uavsar_directory_five
    )

    uavsar_variables_five = {
        'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_1',
        'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_2',
        'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_3',
    }

    expected_uavsar_results_five = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 3,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_five.tiff',
        'spatial_extent': [39.95, 40.0, -123.69, -123.59],
        'variables': uavsar_variables_five,
        'width': 1800,
        'height': 900,
    }

    check_request_output(uavsar_output_five[0], expected_uavsar_results_five)
    clean_test_artefacts(uavsar_directory_five)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 6:

This test performs a bounding box spatial subset that crosses the Equator, while explicitly requesting all variables by identifier "all".

Each band in the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") so the output is expected to have 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1232482059-ASF', 'G1244144732-EEDTEST'
    )

    uavsar_directory_six = 'uavsar_test_six'

    uavsar_request_six = Request(
        collection=environment_information['uavsar_collection'],
        variables=['all'],
        spatial=BBox(11.57, -0.03, 11.60, 0.02),
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_six = make_request_and_download_result(
        harmony_client, uavsar_request_six, uavsar_directory_six
    )

    uavsar_base_six = (
        'eighty_14047_16008_006_160225_L090_CX_01_pauli_subsetted__all__subsetted'
    )

    uavsar_variables_six = {
        f'{uavsar_base_six}_band_1',
        f'{uavsar_base_six}_band_2',
        f'{uavsar_base_six}_band_3',
    }

    expected_uavsar_results_six = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 3,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_six.tiff',
        'spatial_extent': [-0.03, 0.02, 11.57, 11.60],
        'variables': uavsar_variables_six,
        'width': 540,
        'height': 900,
    }

    check_request_output(uavsar_output_six[0], expected_uavsar_results_six)
    clean_test_artefacts(uavsar_directory_six)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 7:

This test performs a bounding box spatial subset in the Northern Hemisphere, explicitly requesting a bounding box that is larger than the input granule extent. To limit the size of the reference image stored in the repository, the request specifies a variable subset for only `Band1`, and the output dimensions are specified to be coarser resolution than the input granule.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1402334529-ASF', 'G1244144730-EEDTEST'
    )

    uavsar_directory_seven = 'uavsar_test_seven'

    uavsar_request_seven = Request(
        collection=environment_information['uavsar_collection'],
        variables=['Band1'],
        spatial=BBox(-139, 61, -137, 62),
        format='image/tiff',
        height=870,
        width=1129,
        granule_id=granule_id,
    )

    uavsar_output_seven = make_request_and_download_result(
        harmony_client, uavsar_request_seven, uavsar_directory_seven
    )

    uavsar_variables_seven = {
        'kluane_01902_17064_005_170615_L090_CX_01_pauli_Band1_regridded_subsetted__Band1__resized_band_1'
    }

    expected_uavsar_results_seven = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_seven.tiff',
        'spatial_extent': [61.57, 61.76, -138.36, -137.85],
        'variables': uavsar_variables_seven,
        'width': 1129,
        'height': 870,
    }

    check_request_output(uavsar_output_seven[0], expected_uavsar_results_seven)
    clean_test_artefacts(uavsar_directory_seven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 8:

This test combines a bounding box spatial subset with a variable subset (`Band2`). The bounding box extends past the western bound of the input granule extent.


In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1223713114-ASF', 'G1244144731-EEDTEST'
    )

    uavsar_directory_eight = 'uavsar_test_eight'

    uavsar_request_eight = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-72.3, -40.02, -72.0, -40.0),
        variables=['Band2'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_eight = make_request_and_download_result(
        harmony_client, uavsar_request_eight, uavsar_directory_eight
    )

    uavsar_variables_eight = {
        'ChiVol_00700_15030_010_150330_L090_CX_01_pauli_Band2_subsetted__Band2__subsetted_band_1'
    }

    expected_uavsar_results_eight = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_eight.tiff',
        'spatial_extent': [-40.02, -40.0, -72.17, -72.0],
        'variables': uavsar_variables_eight,
        'width': 3004,
        'height': 360,
    }

    check_request_output(uavsar_output_eight[0], expected_uavsar_results_eight)
    clean_test_artefacts(uavsar_directory_eight)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 9:

This test combines a bounding box spatial subset with a temporal subset. All three variables are explicitly listed in the request. The temporal subset should return only a single granule, and apply the spatial and variable subset to that granule.

In [ ]:
if environment_information is not None:
    uavsar_directory_nine = 'uavsar_test_nine'

    uavsar_request_nine = Request(
        collection=environment_information['uavsar_collection'],
        variables=['all'],
        spatial=BBox(-89.02, 28.98, -89.00, 29.00),
        temporal={
            'start': datetime(2016, 11, 17, 23, 27, 0),
            'stop': datetime(2016, 11, 17, 23, 28, 0),
        },
        format='image/tiff',
    )

    uavsar_output_nine = make_request_and_download_result(
        harmony_client, uavsar_request_nine, uavsar_directory_nine
    )

    uavsar_variables_nine = {
        'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__all__subsetted_band_1',
        'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__all__subsetted_band_2',
        'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__all__subsetted_band_3',
    }

    expected_uavsar_results_nine = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 3,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_nine.tiff',
        'spatial_extent': [28.98, 29.00, -89.02, -89.00],
        'variables': uavsar_variables_nine,
        'width': 360,
        'height': 360,
    }

    check_request_output(uavsar_output_nine[0], expected_uavsar_results_nine)
    clean_test_artefacts(uavsar_directory_nine)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_one = 'uavsar_test_one'

    uavsar_request_one = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        format='image/tiff',
        granule_id=granule_id,
    )

    uavsar_output_one = make_request_and_download_result(
        harmony_client, uavsar_request_one, uavsar_directory_one
    )

    uavsar_variable_base_one = 'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band1_subsetted__Band1__subsetted_band'

    uavsar_variables_one = {f'{uavsar_variable_base_one}_1'}

    expected_uavsar_results_one = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_one.tiff',
        'spatial_extent': [64.0, 64.1, -145.75, -145.7],
        'variables': uavsar_variables_one,
        'width': 450,
        'height': 1800,
    }

    check_request_output(uavsar_output_one[0], expected_uavsar_results_one)
    clean_test_artefacts(uavsar_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 10:

This test combines a bounding box spatial subset with a variable subset (`Band1`). No format is specified, so the output should be a GeoTIFF.

Note - the test results for this test are the same as UAVSAR test-case 1, and so the reference file is reused from that test.

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_ten = 'uavsar_test_ten'

    uavsar_request_ten = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        granule_id=granule_id,
    )

    uavsar_output_ten = make_request_and_download_result(
        harmony_client,
        uavsar_request_ten,
        uavsar_directory_ten,
    )

    uavsar_variable_base_ten = 'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band1_subsetted__Band1__subsetted_band'

    uavsar_variables_ten = {f'{uavsar_variable_base_ten}_1'}

    expected_uavsar_results_ten = {
        'authority': 'EPSG',
        'cs': 'Geographic',
        'gcs': 'WGS 84',
        'gcs_epsg': '4326',
        'n_bands': 1,
        'proj_cs': None,
        'proj_epsg': None,
        'reference_image': 'reference_images/uavsar_reference_one.tiff',
        'spatial_extent': [64.0, 64.1, -145.75, -145.7],
        'variables': uavsar_variables_ten,
        'width': 450,
        'height': 1800,
    }

    check_request_output(uavsar_output_ten[0], expected_uavsar_results_ten)
    clean_test_artefacts(uavsar_directory_ten)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 11:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2. A netCDF4 output format is specified.

**Note: earthdata-hashdiff comparison is disabled because it requires Python 3.11, and that is incompatible with the version of GDAL used for GeoTIFF comparisons. This test is retained to ensure the request runs without error.**

In [ ]:
if environment_information is not None:
    granule_id = get_test_granule_id(
        environment_information['env'], 'G1422449017-ASF', 'G1244144733-EEDTEST'
    )

    uavsar_directory_eleven = 'uavsar_test_eleven'

    uavsar_request_eleven = Request(
        collection=environment_information['uavsar_collection'],
        spatial=BBox(-145.75, 64.0, -145.70, 64.1),
        variables=['Band1'],
        format='application/x-netcdf4',
        granule_id=granule_id,
    )

    uavsar_output_eleven = make_request_and_download_result(
        harmony_client,
        uavsar_request_eleven,
        uavsar_directory_eleven,
    )

    # assert nc4_matches_reference_hash_file(
    #     uavsar_output_eleven[0], 'reference_images/uavsar_reference_eleven.nc4'
    # ), 'Test output does not match reference file, UAVSAR netCDF4 output'
    clean_test_artefacts(uavsar_directory_eleven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')